In [1]:
import tensorflow as tf
import magenta
import os

In [2]:
sess=tf.InteractiveSession()

In [3]:
#加这行是因为jupyter notebook对tf.app.flags.FLAGS有bug
#见https://github.com/tensorflow/tensorflow/issues/17702
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [4]:
import magenta.models.melody_rnn.melody_rnn_config_flags as config_flags

config_flags.FLAGS.config = 'basic_rnn'
config_flags.FLAGS.hparams = 'batch_size=10,rnn_layer_sizes=[128,128]'

In [5]:
from magenta.models.melody_rnn import melody_rnn_config_flags
config = melody_rnn_config_flags.config_from_flags()

In [6]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('run_dir', r'E:\Magenta\Modify\melody_rnn\weight_jazz',
                           'Path to the directory where checkpoints and '
                           'summary events will be saved during training and '
                           'evaluation. Separate subdirectories for training '
                           'events and eval events will be created within '
                           '`run_dir`. Multiple runs can be stored within the '
                           'parent directory of `run_dir`. Point TensorBoard '
                           'to the parent directory of `run_dir` to see all '
                           'your runs.')
tf.app.flags.DEFINE_string('sequence_example_file_1', r'E:\Magenta\Dataset\melody_rnn_bach\training_melodies.tfrecord',
                           'Path to first dataset TFRecord file containing '
                           'tf.SequenceExample records for training or '
                           'evaluation. A filepattern may also be provided, '
                           'which will be expanded to all matching files.')
tf.app.flags.DEFINE_string('sequence_example_file_2', r'E:\Magenta\Dataset\melody_rnn_jazz_single\training_melodies.tfrecord',
                           'Path to second dataset TFRecord file containing '
                           'tf.SequenceExample records for training or '
                           'evaluation. A filepattern may also be provided, '
                           'which will be expanded to all matching files.')
tf.app.flags.DEFINE_integer('num_training_steps', 0,
                            'The the number of global training steps your '
                            'model should take before exiting training. '
                            'Leave as 0 to run until terminated manually.')
tf.app.flags.DEFINE_integer('num_eval_examples', 0,
                            'The number of evaluation examples your model '
                            'should process for each evaluation step.'
                            'Leave as 0 to use the entire evaluation set.')
tf.app.flags.DEFINE_integer('summary_frequency', 100,
                            'A summary statement will be logged every '
                            '`summary_frequency` steps during training or '
                            'every `summary_frequency` seconds during '
                            'evaluation.')
tf.app.flags.DEFINE_integer('num_checkpoints', 10,
                            'The number of most recent checkpoints to keep in '
                            'the training directory. Keeps all if 0.')
tf.app.flags.DEFINE_boolean('eval', False,
                            'If True, this process only evaluates the model '
                            'and does not update weights.')
tf.app.flags.DEFINE_boolean('condition', True,
                            'If True, will use two datasets above and using condition input.')
tf.app.flags.DEFINE_string('log', 'INFO',
                           'The threshold for what messages will be logged '
                           'DEBUG, INFO, WARN, ERROR, or FATAL.')

In [7]:
tf.logging.set_verbosity(FLAGS.log)

if not FLAGS.run_dir:
    tf.logging.fatal('--run_dir required')
if not FLAGS.sequence_example_file_1:
    tf.logging.fatal('--sequence_example_file_1 required')
if (not FLAGS.sequence_example_file_2) and FLAGS.condition:
    tf.logging.fatal('--sequence_example_file_2 required')
# If use conditioning, add the second example_file to the list
if FLAGS.condition:
    sequence_example_file_paths = (tf.gfile.Glob(
        os.path.expanduser(FLAGS.sequence_example_file_1))
                                   + tf.gfile.Glob(
                os.path.expanduser(FLAGS.sequence_example_file_2)))
else:
    sequence_example_file_paths = tf.gfile.Glob(
        os.path.expanduser(FLAGS.sequence_example_file_1))
run_dir = os.path.expanduser(FLAGS.run_dir)

In [8]:
print(sequence_example_file_paths)
print(type(sequence_example_file_paths))

['E:\\Magenta\\Dataset\\melody_rnn_bach_single\\training_melodies.tfrecord', 'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord']
<class 'list'>


In [9]:
config = melody_rnn_config_flags.config_from_flags()

mode = 'eval' if FLAGS.eval else 'train'

In [10]:
hparams = config.hparams
encoder_decoder = config.encoder_decoder
tf.logging.info('hparams = %s', hparams.values())

input_size = encoder_decoder.input_size
num_classes = encoder_decoder.num_classes
no_event_label = encoder_decoder.default_event_label

INFO:tensorflow:hparams = {'dropout_keep_prob': 0.5, 'batch_size': 10, 'clip_norm': 5, 'use_cudnn': False, 'rnn_layer_sizes': [128, 128], 'attn_length': 0, 'residual_connections': False, 'learning_rate': 0.001}


In [11]:
import numbers
from magenta.models.shared.events_rnn_graph import *
inputs, labels, lengths = None, None, None

In [12]:
if mode == 'train' or mode == 'eval':
    if isinstance(no_event_label, numbers.Number):
        label_shape = []
    else:
        label_shape = [len(no_event_label)]


In [13]:
'''
inputs, labels, lengths = magenta.common.get_padded_batch(
          sequence_example_file_paths, hparams.batch_size, input_size,
          label_shape=label_shape, shuffle=mode == 'train')
'''
file_list=sequence_example_file_paths
batch_size=hparams.batch_size
num_enqueuing_threads=4
#shuffle=mode == 'train'
shuffle=False

In [14]:
import math
from magenta.common.sequence_example_lib import *
def _shuffle_inputs(input_tensors, capacity, min_after_dequeue, num_threads):
    """Shuffles tensors in `input_tensors`, maintaining grouping."""
    shuffle_queue = tf.RandomShuffleQueue(
        capacity, min_after_dequeue, dtypes=[t.dtype for t in input_tensors])
    enqueue_op = shuffle_queue.enqueue(input_tensors)
    runner = tf.train.QueueRunner(shuffle_queue, [enqueue_op] * num_threads)
    tf.train.add_queue_runner(runner)

    output_tensors = shuffle_queue.dequeue()

    for i in range(len(input_tensors)):
        output_tensors[i].set_shape(input_tensors[i].shape)

    return output_tensors

In [15]:
file_queue = tf.train.string_input_producer(file_list)
reader = tf.TFRecordReader()
read_queue, serialized_example = reader.read(file_queue)

sequence_features = {
    'inputs': tf.FixedLenSequenceFeature(shape=[input_size],
                                         dtype=tf.float32),
    'labels': tf.FixedLenSequenceFeature(shape=label_shape or [],
                                         dtype=tf.int64)}

single_queue, sequence = tf.parse_single_sequence_example(
    serialized_example, sequence_features=sequence_features)


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [ ]:
'''file_queue = tf.train.string_input_producer(file_list)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(file_queue)

sequence_features = {
    'inputs': tf.FixedLenSequenceFeature(shape=[input_size],
                                         dtype=tf.float32),
    'labels': tf.FixedLenSequenceFeature(shape=label_shape or [],
                                         dtype=tf.int64)}

_, sequence = tf.parse_single_sequence_example(
    serialized_example, sequence_features=sequence_features)

length = tf.shape(sequence['inputs'])[0]
input_tensors = [sequence['inputs'], sequence['labels'], length]

if shuffle:
    if num_enqueuing_threads < 2:
        raise ValueError(
            '`num_enqueuing_threads` must be at least 2 when shuffling.')
    shuffle_threads = int(math.ceil(num_enqueuing_threads) / 2.)

    # Since there may be fewer records than SHUFFLE_MIN_AFTER_DEQUEUE, take the
    # minimum of that number and the number of records.
    min_after_dequeue = count_records(
        file_list, stop_at=SHUFFLE_MIN_AFTER_DEQUEUE)
    input_tensors = _shuffle_inputs(
        input_tensors, capacity=QUEUE_CAPACITY,
        min_after_dequeue=min_after_dequeue,
        num_threads=shuffle_threads)

    num_enqueuing_threads -= shuffle_threads

tf.logging.info(input_tensors)
'''

# 看原始输出

In [22]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)
f=0
try:
    for i in range (10):
        seq=sess.run(sequence)
        # print(np.argmax(inp['inputs'],axis=1).shape)
        print(seq['inputs'].shape)
        print(seq['labels'].shape)
        #print(type(seq))
except tf.errors.OutOfRangeError:
    print("done")
finally:
    coord.request_stop()
#coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, FIFOQueue '_1_input_producer' is closed.
	 [[{{node input_producer/input_producer_EnqueueMany}} = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/RandomShuffle)]]
(303, 38)
(303,)
(303, 38)
(303,)
(303, 38)
(303,)
(303, 38)
(303,)
(303, 38)
(303,)
(511, 38)
(511,)
(111, 38)
(111,)
(511, 38)
(511,)
(502, 38)
(502,)
(502, 38)
(502,)


In [17]:
inp=np.argmax(seq['inputs'],axis=1)


In [18]:
inp

array([ 0,  0,  0,  0, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 19,
        0,  0,  0,  0,  0,  0,  1, 18,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0, 18,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0, 15,  0,  0,
        0,  0,  0,  0,  1, 15,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 14,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,
        0,  0,  1, 13,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 12,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 11,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int64)

In [20]:
seq['labels']

array([ 0,  0,  0, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 19,  0,
        0,  0,  0,  0,  0,  1, 18,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
       18,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0, 15,  0,  0,  0,
        0,  0,  0,  1, 15,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 14,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,
        0,  1, 13,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 12,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 11,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int64)

In [15]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)
f=0
try:
    for i in range (10):
        que=sess.run(read_queue)
        # print(np.argmax(inp['inputs'],axis=1).shape)
        print(que)
        print(type(que))
except tf.errors.OutOfRangeError:
    print("done")
finally:
    coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
b'E:\\Magenta\\Dataset\\melody_rnn_bach_single\\training_melodies.tfrecord:0'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_bach_single\\training_melodies.tfrecord:50954'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_bach_single\\training_melodies.tfrecord:101908'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_bach_single\\training_melodies.tfrecord:152862'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_bach_single\\training_melodies.tfrecord:203816'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord:0'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord:85898'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord:104596'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord:190494'
<class 'bytes'>
b'E:\\Magenta\\Dataset\\melody_rnn_jaz

In [18]:
que

b'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord:700756'

In [19]:
str(que)

"b'E:\\\\Magenta\\\\Dataset\\\\melody_rnn_jazz_single\\\\training_melodies.tfrecord:700756'"

In [22]:
str_que=str(que)
str_que.find('bach')

-1

In [30]:
type(que) is list

True

In [28]:
a=type(que)

In [29]:
type(a)

type

In [ ]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess, coord)
f=0
try:
    seq_list=[]
    for i in range (100):
        seq=sess.run(sequence)
        seq_list.append((seq['inputs'].shape[0]))
        #print('inputs: '+str(seq['inputs'].shape)+' labels:'+str(seq['labels'].shape))
        if i %500 ==0:
            print(i)
except tf.errors.OutOfRangeError:
    print("done")
finally:
    coord.request_stop()
coord.join(threads)

In [ ]:
seq_list

In [ ]:
l_len=10
f=0
for i in range(1,1990):
    for index in range(10):
        if seq_list[index+i]!= seq_list[index]:
            f-=1
        f+=1
    if f==10:
        print('Found: '+str(i))
    else:
        f=0



In [ ]:
seq_list

In [ ]:

        for index in range(10):
            if seq_list[index-10]!= seq_list[index]:
                f-=1
            f+=1
            print(f)
            if f==10:
                print('Found: '+str(i))
            else:
                f=0

In [ ]:
len(seq_list)

In [ ]:
print(seq_list[0:10])
print(seq_list[1513:1513+10])

WARNING:tensorflow:From <ipython-input-15-da84dd1d818a>:2: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
inputs: (351, 38) labels:(351,)
inputs: (175, 38) labels:(175,)
inputs: (175, 38) labels:(175,)
inputs: (239, 38) labels:(239,)
inputs: (239, 38) labels:(239,)
inputs: (211, 38) labels:(211,)
inputs: (359, 38) labels:(359,)
inputs: (159, 38) labels:(159,)
inputs: (159, 38) labels:(159,)
inputs: (307, 38) labels:(307,)

In [ ]:
seq

In [ ]:
len(seq)

In [9]:
sequence_example_file_paths

['E:\\Magenta\\Dataset\\melody_rnn_bach\\training_melodies.tfrecord',
 'E:\\Magenta\\Dataset\\melody_rnn_jazz_single\\training_melodies.tfrecord']

In [10]:
file_path=sequence_example_file_paths[0]

In [11]:
file_path

'E:\\Magenta\\Dataset\\melody_rnn_bach\\training_melodies.tfrecord'

In [ ]:
c = 0
for fn in file_path:
    for record in tf.python_io.tf_record_iterator(fn):
        c += 1
    print(c)

In [13]:
c

0

ERROR! Session/line number was not unique in database. History logging moved to new session 539
